In [86]:
#Bibliotecas
import librosa
import itertools
import numpy as np
import pandas as pd
import warnings
from os import listdir
from os import chdir
from sklearn.cluster import KMeans  
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import datetime

In [87]:
warnings.filterwarnings("ignore")
print(datetime.datetime.now())

2018-08-17 23:15:15.209118


In [88]:
def pontosSegment(db,i):
    while True:
        try:
            bounds = librosa.segment.agglomerative(db, i)
            break
        except:
            i = i - 1
    return bounds

In [89]:
def SubSegmentacao(audio,fs,tamanho,tempo,pontos):
    i = pontos
    D = np.abs(librosa.stft(audio))**2
    S = librosa.feature.melspectrogram(S=D)
    db = librosa.amplitude_to_db(S)
    bounds = pontosSegment(db,i)
    bound_times = librosa.frames_to_time(bounds, sr=fs)
    inicio = int(tamanho*(np.min(bound_times[1:])/tempo))
    fim    = int(tamanho*(np.max(bound_times[1:])/tempo))
    return (audio[inicio:fim])

In [90]:
def SegmentarAudio(audio,fs,n,tamanho,tempo):
    i = 500
    D = np.abs(librosa.stft(audio))**2
    S = librosa.feature.melspectrogram(S=D)
    db = librosa.amplitude_to_db(S)
    bounds = pontosSegment(db,i)
    bound_times = librosa.frames_to_time(bounds, sr=fs)
    bound_times = np.vstack((bound_times[2:], [0]*len(bound_times[2:]))).T
    kmeans = KMeans(n_clusters=n)  
    kmeans.fit(bound_times[2:])  
    func1 = lambda x: int(tamanho*(x/tempo))
    centros =  np.array([func1(xi) for xi in kmeans.cluster_centers_[:,0]])
    centros =  np.append(centros,tamanho)
    centros.sort()
    ultimo  = 0
    retorno = []
    for i,j in zip(centros,centros[1:]):
        sl = int((i+j)/2)
        retorno.append(SubSegmentacao(audio[ultimo:sl],fs,tamanho,tempo,10))
        ultimo = sl
    return np.array(retorno)

In [91]:
scaler      = StandardScaler()
nLetras     = 4   #Número de letras que constará por arquivo
n_mfcc      = 75 #Número de componentes MFCC que irá ser extraido

def ExtrairMFCC(Pasta):
    chdir(Pasta)
    print('Extraindo atributo MFCC dos arquivos...')
    atributos  = []
    rotulos    = []
    arquivos = listdir()
    arquivos.sort()
    for arquivo in arquivos:
        if 'wav' in arquivo:
            audio, fs = librosa.load(arquivo,None)
            tamanho = audio.shape[0]
            tempo   = audio.shape[0]/fs
            retorno = SegmentarAudio(audio, fs, nLetras, tamanho, tempo)
            print('Extraindo MFCC do arquivo ', arquivo, '.')
            for i in range(0, nLetras): 
                retorno[i,] = librosa.util.normalize(retorno[i,],axis=0, norm=1)
                mfcc2    =  librosa.feature.mfcc(retorno[i,],  sr=fs, n_mfcc=n_mfcc,hop_length=64).T
                rec = librosa.segment.recurrence_matrix(mfcc2, mode='affinity',metric='jaccard', sparse=True)
                mfcc2 = librosa.decompose.nn_filter(mfcc2, rec=rec,aggregate=np.average)
                mfcc2 = scaler.fit_transform(mfcc2)
                atributos.append(mfcc2)
                rotulos.append(arquivo[i])
                
    return atributos, rotulos

    

In [92]:
chdir('..')
atributos, rotulos = ExtrairMFCC('Treino')

Extraindo atributo MFCC dos arquivos...
Extraindo MFCC do arquivo  66bx.wav .
Extraindo MFCC do arquivo  66c7.wav .
Extraindo MFCC do arquivo  66cb.wav .
Extraindo MFCC do arquivo  66cc.wav .
Extraindo MFCC do arquivo  67aa.wav .
Extraindo MFCC do arquivo  67bm.wav .
Extraindo MFCC do arquivo  67mc.wav .
Extraindo MFCC do arquivo  67n6.wav .
Extraindo MFCC do arquivo  6a67.wav .
Extraindo MFCC do arquivo  6a7h.wav .
Extraindo MFCC do arquivo  6ad6.wav .
Extraindo MFCC do arquivo  6ahn.wav .
Extraindo MFCC do arquivo  6ama.wav .
Extraindo MFCC do arquivo  6anh.wav .
Extraindo MFCC do arquivo  6c67.wav .
Extraindo MFCC do arquivo  6c6c.wav .
Extraindo MFCC do arquivo  6c7d.wav .
Extraindo MFCC do arquivo  6dbc.wav .
Extraindo MFCC do arquivo  6dmn.wav .
Extraindo MFCC do arquivo  6dnc.wav .
Extraindo MFCC do arquivo  6hab.wav .
Extraindo MFCC do arquivo  6hcm.wav .
Extraindo MFCC do arquivo  6hcx.wav .
Extraindo MFCC do arquivo  6hhb.wav .
Extraindo MFCC do arquivo  6hhd.wav .
Extraindo 

Extraindo MFCC do arquivo  c6b7.wav .
Extraindo MFCC do arquivo  c6bm.wav .
Extraindo MFCC do arquivo  c6d6.wav .
Extraindo MFCC do arquivo  c7am.wav .
Extraindo MFCC do arquivo  c7cx.wav .
Extraindo MFCC do arquivo  c7dd.wav .
Extraindo MFCC do arquivo  c7m6.wav .
Extraindo MFCC do arquivo  c7mh.wav .
Extraindo MFCC do arquivo  caac.wav .
Extraindo MFCC do arquivo  cam6.wav .
Extraindo MFCC do arquivo  camh.wav .
Extraindo MFCC do arquivo  cbcm.wav .
Extraindo MFCC do arquivo  cbha.wav .
Extraindo MFCC do arquivo  cbnm.wav .
Extraindo MFCC do arquivo  cbx7.wav .
Extraindo MFCC do arquivo  cc66.wav .
Extraindo MFCC do arquivo  cc6m.wav .
Extraindo MFCC do arquivo  ccam.wav .
Extraindo MFCC do arquivo  ccb6.wav .
Extraindo MFCC do arquivo  ccdb.wav .
Extraindo MFCC do arquivo  ccdd.wav .
Extraindo MFCC do arquivo  ccdm.wav .
Extraindo MFCC do arquivo  cddb.wav .
Extraindo MFCC do arquivo  cdn6.wav .
Extraindo MFCC do arquivo  cdnx.wav .
Extraindo MFCC do arquivo  cdxc.wav .
Extraindo MF

Extraindo MFCC do arquivo  nhmx.wav .
Extraindo MFCC do arquivo  nhxb.wav .
Extraindo MFCC do arquivo  nm6c.wav .
Extraindo MFCC do arquivo  nmb6.wav .
Extraindo MFCC do arquivo  nmcb.wav .
Extraindo MFCC do arquivo  nmd6.wav .
Extraindo MFCC do arquivo  nmdx.wav .
Extraindo MFCC do arquivo  nmm6.wav .
Extraindo MFCC do arquivo  nmmx.wav .
Extraindo MFCC do arquivo  nmn6.wav .
Extraindo MFCC do arquivo  nmnc.wav .
Extraindo MFCC do arquivo  nna7.wav .
Extraindo MFCC do arquivo  nnd7.wav .
Extraindo MFCC do arquivo  nnh6.wav .
Extraindo MFCC do arquivo  nnhh.wav .
Extraindo MFCC do arquivo  nnhx.wav .
Extraindo MFCC do arquivo  nnmn.wav .
Extraindo MFCC do arquivo  nnnb.wav .
Extraindo MFCC do arquivo  nnxm.wav .
Extraindo MFCC do arquivo  nx6m.wav .
Extraindo MFCC do arquivo  nx76.wav .
Extraindo MFCC do arquivo  nxax.wav .
Extraindo MFCC do arquivo  nxcc.wav .
Extraindo MFCC do arquivo  nxdb.wav .
Extraindo MFCC do arquivo  nxh6.wav .
Extraindo MFCC do arquivo  nxxc.wav .
Extraindo MF

In [77]:
from sklearn.model_selection import StratifiedKFold
import datetime


def ValidacaoCaracter(X, Y):
    skf = StratifiedKFold(n_splits=10, random_state = 72)
    
    for train_index, test_index in skf.split(X, Y):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
            
            y_train = list(map(lambda x,y: np.array(x.shape[0]*[y])  , X_train,y_train ))
            y_test  = list(map(lambda x,y: np.array(x.shape[0]*[y])  , X_test, y_test  ))
            
            X_train   = np.vstack(X_train)
            y_train   = np.hstack(y_train)
            
            modelo = RandomForestClassifier(n_estimators=17,criterion="entropy",n_jobs=-1,random_state = 72)
            modelo.fit(X_train, y_train)
            print('Treinado')
            
            nQuantidade = 0
            nAcertos    = 0
            
            for x,y in zip(X_test, y_test):
                nQuantidade = nQuantidade + 1
                    
                (rot,counts) = np.unique(y,return_counts=True)
                ind1 = np.argmax(counts)
                
                pred     = modelo.predict(x) 
                (values,counts) = np.unique(pred,return_counts=True)
                ind = np.argmax(counts)


                if values[ind] == rot[ind1]:
                    nAcertos = nAcertos + 1
                    #print(rot[ind1])
                    #print(values[ind])
                    
            
            vAcuraciaCAPTCHA = (nAcertos/nQuantidade)
            print("Acurácia por CARACTER: {:.2f}".format(vAcuraciaCAPTCHA*100),"%")





In [78]:
atributos = np.array(atributos)
rotulos   = np.array(rotulos)




ValidacaoCaracter(atributos,rotulos)



Treinado
Acurácia por CARACTER: 78.22 %
Treinado
Acurácia por CARACTER: 79.10 %
Treinado
Acurácia por CARACTER: 77.50 %
Treinado
Acurácia por CARACTER: 80.00 %
Treinado
Acurácia por CARACTER: 82.91 %
Treinado
Acurácia por CARACTER: 78.89 %
Treinado
Acurácia por CARACTER: 79.19 %
Treinado
Acurácia por CARACTER: 82.14 %
Treinado
Acurácia por CARACTER: 80.83 %
Treinado
Acurácia por CARACTER: 78.24 %


In [93]:

def TreinarModelo(atributos, rotulos):
        rotulos = list(map(lambda x,y: np.array(x.shape[0]*[y])  , atributos, rotulos ))
            
        atributos   = np.vstack(atributos)
        rotulos     = np.hstack(rotulos)
        modelo = RandomForestClassifier(n_estimators=17,criterion="entropy",n_jobs=-1,random_state = 72)
        modelo.fit(atributos, rotulos)
        
        return modelo
    
    

In [94]:
atributos = np.array(atributos)
rotulos   = np.array(rotulos)

modelo = TreinarModelo(atributos, rotulos)



In [ ]:
def ValidacaoModelo(modelo, atributos, rotulos):
    lstPreditos = []
    lstReais    = []
    nQuantidade = 0
    nCapCorre   = 0
    nCaracter   = 0
    nAcerto     = 0

    
    for x,y in zip(atributos, rotulos):
        nCaracter = nCaracter + 1
        x   = np.vstack(x)
            
        pred     = modelo.predict(x) 
        (values,counts) = np.unique(pred,return_counts=True)
        ind = np.argmax(counts)
        
        lstPreditos.append(values[ind])
        lstReais.append(y)
        #print(y)
        
        if values[ind] == y:
            nAcerto = nAcerto + 1
            #print(y)
        
        if nCaracter == 4:
            #print('---')
            if nAcerto == 4:
                nCapCorre = nCapCorre + 1
               # print('ok')
            nAcerto   = 0
            nCaracter = 0
            nQuantidade = nQuantidade + 1
            #print('---')
            
    return lstPreditos, lstReais, (nCapCorre/nQuantidade)
                
        
        
    
    

In [ ]:
chdir('..')
atributosVal, rotulosVal = ExtrairMFCC('base_validacao')

Extraindo atributo MFCC dos arquivos...
Extraindo MFCC do arquivo  66dn.wav .
Extraindo MFCC do arquivo  6ah7.wav .
Extraindo MFCC do arquivo  6amx.wav .
Extraindo MFCC do arquivo  6ba6.wav .
Extraindo MFCC do arquivo  6bx7.wav .
Extraindo MFCC do arquivo  6bxd.wav .
Extraindo MFCC do arquivo  6c7a.wav .
Extraindo MFCC do arquivo  6cd6.wav .
Extraindo MFCC do arquivo  6h7n.wav .
Extraindo MFCC do arquivo  6hb6.wav .
Extraindo MFCC do arquivo  6mab.wav .
Extraindo MFCC do arquivo  6mcb.wav .
Extraindo MFCC do arquivo  6nh7.wav .
Extraindo MFCC do arquivo  6nxd.wav .
Extraindo MFCC do arquivo  6xaa.wav .
Extraindo MFCC do arquivo  76m6.wav .
Extraindo MFCC do arquivo  7a7m.wav .
Extraindo MFCC do arquivo  7acm.wav .
Extraindo MFCC do arquivo  7b7c.wav .
Extraindo MFCC do arquivo  7bxm.wav .
Extraindo MFCC do arquivo  7c66.wav .
Extraindo MFCC do arquivo  7cac.wav .
Extraindo MFCC do arquivo  7dc7.wav .
Extraindo MFCC do arquivo  7hah.wav .
Extraindo MFCC do arquivo  7m6b.wav .
Extraindo 

In [83]:
lstPreditos = []
lstReais    = []
vAcuraciaCAPTCHA   = 0.0



lstPreditos, lstReais, vAcuraciaCAPTCHA  = ValidacaoModelo(modelo, atributosVal, rotulosVal)


print("Acurácia por CAPTCHA: {:.2f}".format(vAcuraciaCAPTCHA*100),"%")


Acurácia por CAPTCHA: 30.20 %


In [84]:
Areais    =  np.array(lstReais)
Apreditos =  np.array(lstPreditos)
print ("Matriz de Confusão\n")
print (pd.crosstab(Areais,Apreditos,rownames=['REAL'],colnames=['PREDITO'],margins=True))

Matriz de Confusão

PREDITO   6   7   a    b   c   d   h   m   n   x  All
REAL                                                 
6        61   3   1    3   0   0   0   0   2   0   70
7         0  62   0    1   0   0   0   0   0   0   63
a         0   1  61    0   0   0   2   0   0   0   64
b         0   2   0   46   0   3   0   0   2   0   53
c        18   4   1    4  35   0   0   1   2   1   66
d         1   0   2   32   0  16   0   0   4   0   55
h         0   0   3    0   0   0  53   0   0   0   56
m         0   1   0    6   0   0   1  33  16   0   57
n         0   2   0   10   0   0   0   6  27   0   45
x        10   0   1    1   0   0   0   0   2  53   67
All      90  75  69  103  35  19  56  40  55  54  596


In [85]:
print(datetime.datetime.now())


2018-08-17 20:51:45.818892
